In [283]:
import importlib
import SpotifyAPIClient
importlib.reload(SpotifyAPIClient)
from SpotifyAPIClient import SpotifyAPI

client_id = 'd7a2e6f4a8434550baa5eda073f0a6a3'
client_secret = 'def46d47ba584e378b7667645666a468'

In [284]:
spotify = SpotifyAPI(client_id, client_secret)
spotify.get_user_playlist('armandov10', 1,0)

{'href': 'https://api.spotify.com/v1/users/armandov10/playlists?offset=0&limit=1',
 'items': [{'collaborative': False,
   'description': '',
   'external_urls': {'spotify': 'https://open.spotify.com/playlist/4ZxkrfyafRtJoPh4hHNNsG'},
   'href': 'https://api.spotify.com/v1/playlists/4ZxkrfyafRtJoPh4hHNNsG',
   'id': '4ZxkrfyafRtJoPh4hHNNsG',
   'images': [{'height': 640,
     'url': 'https://mosaic.scdn.co/640/ab67616d0000b273249f2b94cd3bd266a0f4a947ab67616d0000b27335a44d80c5b8a026626d6804ab67616d0000b27380203a511b5e24945f719808ab67616d0000b273cac7a5726adb721d54d4c8ca',
     'width': 640},
    {'height': 300,
     'url': 'https://mosaic.scdn.co/300/ab67616d0000b273249f2b94cd3bd266a0f4a947ab67616d0000b27335a44d80c5b8a026626d6804ab67616d0000b27380203a511b5e24945f719808ab67616d0000b273cac7a5726adb721d54d4c8ca',
     'width': 300},
    {'height': 60,
     'url': 'https://mosaic.scdn.co/60/ab67616d0000b273249f2b94cd3bd266a0f4a947ab67616d0000b27335a44d80c5b8a026626d6804ab67616d0000b27380203a5

In [285]:
spotify.get_playlist('4ZxkrfyafRtJoPh4hHNNsG')

{'collaborative': False,
 'description': '',
 'external_urls': {'spotify': 'https://open.spotify.com/playlist/4ZxkrfyafRtJoPh4hHNNsG'},
 'followers': {'href': None, 'total': 0},
 'href': 'https://api.spotify.com/v1/playlists/4ZxkrfyafRtJoPh4hHNNsG',
 'id': '4ZxkrfyafRtJoPh4hHNNsG',
 'images': [{'height': 640,
   'url': 'https://mosaic.scdn.co/640/ab67616d0000b273249f2b94cd3bd266a0f4a947ab67616d0000b27335a44d80c5b8a026626d6804ab67616d0000b27380203a511b5e24945f719808ab67616d0000b273cac7a5726adb721d54d4c8ca',
   'width': 640},
  {'height': 300,
   'url': 'https://mosaic.scdn.co/300/ab67616d0000b273249f2b94cd3bd266a0f4a947ab67616d0000b27335a44d80c5b8a026626d6804ab67616d0000b27380203a511b5e24945f719808ab67616d0000b273cac7a5726adb721d54d4c8ca',
   'width': 300},
  {'height': 60,
   'url': 'https://mosaic.scdn.co/60/ab67616d0000b273249f2b94cd3bd266a0f4a947ab67616d0000b27335a44d80c5b8a026626d6804ab67616d0000b27380203a511b5e24945f719808ab67616d0000b273cac7a5726adb721d54d4c8ca',
   'width': 60}]